First, import some packages that we will need.

Note that we import roka_bundled (i.e., the bundled version of the roka package that makes several parts of this calculation more convenient). I used the bundled version to make sure that the results are reproducible even if you have a more recent version of roka installed on your system.

In [20]:
import numpy as np
import pandas as pd

import roka_bundled as roka

# printing nice DataFrames
from roka_bundled import show_df

# calibration helper functions
from roka_bundled import add_basic_features

# units
from roka_bundled import mm, mm2, MPa, GPa

# classes for FEM analysis
from roka_bundled import DemandProtocol, Steel4, BRB, Analysis

## Candidate models
Create a DataFrame with the parameters of the candidate models.
In this example, we are working with a bilinear BRB model. We assume that E_0 and f_y are identified with sufficiently high accuracy, and the two unknown parameters are b_k (kinematic hardening) and b_i (isotropic hardening). Because the parameter b_i influences both cyclic and inelastic hardening, we will use b_pl = b_k + b_i for inelastic hardening to include both effects.

In [6]:
# specify the domain of interest
b_pl_range = [0.006, 0.06]
b_i_range = [0.0006, 0.006]

# specify the level of discretization
# 3 bins allow quick testing of this notebook
# for serious calculations I suggest using at least 20 bins in each dimension
b_pl_bins = 3
b_i_bins = 3

# generate the samples
b_pl_list = np.logspace(np.log10(b_pl_range[0]), np.log10(b_pl_range[1]), num=b_pl_bins)
b_i_list = np.logspace(np.log10(b_i_range[0]), np.log10(b_i_range[1]), num=b_i_bins)
b_pl, b_i = np.meshgrid(b_pl_list, b_i_list)
b_pl = b_pl.flatten()
b_i = b_i.flatten()

# get the value of b_k from b_pl given b_i
b_k = b_pl - b_i

# create the DataFrame for the candidates (note that we save the hyperparameter b_pl as well)
candidates = pd.DataFrame({'b_pl':b_pl, 'b_k': b_k, 'b_i':b_i})
candidates.index = candidates.index+1 #this is to make the index 1-based

# take a quick look at the values to check if everything is OK
show_df(candidates.head(5))
show_df(candidates.tail(5))

# save the candidates to an HDF5 file
candidates.to_hdf('candidates.h5','data',mode='w')

,b_pl,b_k,b_i
1,0.006000,0.005400,0.000600
2,0.018974,0.018374,0.000600
3,0.060000,0.059400,0.000600
4,0.006000,0.004103,0.001897
5,0.018974,0.017076,0.001897


,b_pl,b_k,b_i
5,0.018974,0.017076,0.001897
6,0.060000,0.058103,0.001897
7,0.006000,0.000000,0.006000
8,0.018974,0.012974,0.006000
9,0.060000,0.054000,0.006000


## Calibration methods

### Force-based (curve-fitting)

In [7]:
def calibration_force(reference, simulation):
    """
    Calculates the calibration error using the force-based, curve-fitting approach by minimizing the root-mean-squared error between the reference and simulation responses.
    
    Parameters
    ----------
    reference
    
    simulation

    Returns
    -------
    eps_CAL

    """
    
    # calculate the error in force response
    eps = reference['F'] - simulation['F']
    
    # the calibration error is defined as the root-mean-squared error
    eps_CAL = np.sqrt(np.mean(eps**2.0))
    
    return eps_CAL

### Stiffness-hardening based (BRB-specific)

In [8]:
def get_SH_features(response, K_lim=None):
    """
    Calculates additional features needed for stiffness-hardening calibration. 
    
    Assumes that the response data include characteristics extracted using the 
    calc_characteristics method.
    
    Parameters
    ----------
    response: DataFrame

    Returns
    -------
    SH_features: Series

    """
    features = dict()

    # stiffness features are based on a subset of the response where tangent 
    # stiffness is below a certain threshold
    if K_lim is not None:
        K_list = response.index[response['K'].abs() < K_lim]
    else:
        K_list = response.index

    # prepare two subsets to differentiate between positive and negative direction loading
    K_plus = response.loc[K_list, 'load_dir'] == 1
    K_minus = response.loc[K_list, 'load_dir'] == -1

    # extract the features
    features.update(dict([        
        ('K_pos', np.mean(response.loc[K_list, 'K'][K_plus])),
        ('K_neg', np.mean(response.loc[K_list, 'K'][K_minus])),
        ('F_pos', np.mean(response.loc[K_list, 'F'][K_plus])),
        ('F_neg', np.mean(response.loc[K_list, 'F'][K_minus])),        
    ]))

    return pd.Series(features)  

def calibration_stiffness_hardening(reference_features, simulation_features):
    """
    Calculates the calibration error using the stiffness-hardening-based approach. 
    
    Assumes that both the reference and the simulation features are from the get_SH_features
    method.
    
    Parameters
    ----------
    reference_features: Series
    
    simulation_features: Series

    Returns
    -------
    eps_CAL

    """
    
    eps_CAL_S = np.log((simulation_features['K_pos']-simulation_features['K_neg']) / 
                   (reference_features['K_pos']-reference_features['K_neg']))
    eps_CAL_H = np.log((simulation_features['F_pos']-simulation_features['F_neg']) / 
                   (reference_features['F_pos']-reference_features['F_neg']))
    
    eps_CAL = 2.0 * np.abs(eps_CAL_S) + np.abs(eps_CAL_H)
    
    return eps_CAL    

## Quasi-static experiment

In [11]:
# uncomment the one you want to use

# Standard load protocol
#eps_list = np.array([ 
#    1., -1., 2., -2.,
#    5., -5., 5., -5.,
#    10.,-10.,10.,-10.,
#    15.,-15.,15.,-15.,
#    20.,-20.,20.,-20.,
#    20.,-20.,20.,-20.,
#    20.,-20.,20.,-20.,
#    20.,-20.,20.,-20.])*mm

# Pulse protocol
eps_list = np.array([
    1., -1.,  2., -2.,
    30.,-30.])*mm

dp = DemandProtocol(eps_list, )

print(dp.demand_list)

[ 0.001 -0.001  0.002 -0.002  0.03  -0.03 ]


## Reference component & results
### Create the reference BRB element

In [12]:
reference_component = BRB(1, 
                          l_tot=1.000, 
                          A_y=1000. * mm2, 
                          f_SM=1.00, 
                          f_DM=1.00, 
                          f_yd=235*MPa, 
                          gamma_ov=1.20)

### Run the virtual quasi-static experiment

In [15]:
# set the step size to a sufficiently small value to follow the transition from elastic to inelastic behavior
dp.step_size = eps_y/50.

# perform the virtual quasi-static test
ref_response = Analysis().material_response(reference_material, dp)

# calculate the normalized forces and displacements
reference_material = reference_component.material
ref_response['d'] = ref_response['eps'] / reference_material.eps_y
ref_response['F'] = ref_response['sig'] / reference_material.f_y

# plot some stats of the reference response to quickly check if everything seems ok
show_df(ref_response.describe([0.1, 0.5, 0.9]))

,eps,sig,d,F
count,3802.000000,3.802000e+03,3802.000000,3802.000000
mean,0.004400,-7.713630e+07,3.276363,-0.273533
std,0.015675,2.847829e+08,11.672977,1.009868
min,-0.030000,-4.452707e+08,-22.340426,-1.578974
10%,-0.019817,-4.039875e+08,-14.757574,-1.432580
50%,0.004479,-1.842007e+08,3.335532,-0.653194
90%,0.024896,3.150614e+08,18.539447,1.117239
max,0.030000,3.312128e+08,22.340426,1.174514


### Get the features used in the relevant calibration

In [19]:
# extract some basic features from the F-d response data
ref_response = add_basic_features(ref_response)
show_df(ref_response.head())

# get the stiffness-hardening features
ref_features = get_SH_features(ref_response, K_lim=0.1)
show_df(ref_features)

,eps,sig,d,F,d_c,half_cycle,load_dir,K,E,E_c
0,0.000000,0.000000e+00,0.00,0.00,0.00,0,1.0,1.0,0.0000,0.0000
1,0.000027,5.640000e+06,0.02,0.02,0.02,0,1.0,1.0,0.0002,0.0002
2,0.000054,1.128000e+07,0.04,0.04,0.04,0,1.0,1.0,0.0008,0.0008
3,0.000081,1.692000e+07,0.06,0.06,0.06,0,1.0,1.0,0.0018,0.0018
4,0.000107,2.256000e+07,0.08,0.08,0.08,0,1.0,1.0,0.0032,0.0032


,0
K_pos,0.011731
K_neg,-0.027957
F_pos,1.090133
F_neg,-1.090408


## Simulation components & results

In [22]:
def make_bilin_material(b_k, b_i, f_y, E_0, **kwargs):
    
    material = Steel4(
        1, non_sym=False, kin=True, iso=True, ult=False,
        f_y=f_y, E_0=E_0, b_k=b_k, b_i=b_i,
        # the following parameters are only needed to make the response bilinear
        R_0=50., r_1=0.05, rho_i=1.0, b_l=b_i, l_yp=0., R_u=50.)
    
    return material

### Calibration error in quasi-static tests

In [28]:
# load the parameters of the candidate models (we saved them earlier)
candidates = pd.read_hdf('candidates.h5','data')

# initialize the database to store the calibration error values
eps_CAL_df = pd.DataFrame(columns=['eps_CAL_F','eps_CAL_SH'], 
                          index=candidates.index, dtype=np.float)

# load the basic parameters of the reference material
# we will use these to normalize the F-d response below
f_y_ref = reference_material.f_y
eps_y_ref = reference_material.eps_y
E_0_ref = reference_material.E_0

# perform the simulation of the quasi-static test with each model
for sim_id in candidates.index.values:
    
    print(sim_id, end = ' ')
    
    # create the material using the candidate model's parameters
    sim_material = make_bilin_material(**candidates.loc[sim_id], 
                                       f_y=f_y_ref, E_0=E_0_ref)
    
    # note that the step size for dp has already been set earlier
    # keeping the same step size facilitates the comparison of reference and simulation data
    sim_response = Analysis().material_response(sim_material, dp)
    
    # normalize the response
    sim_response['d'] = sim_response['eps'] / eps_y_ref
    sim_response['F'] = sim_response['sig'] / f_y_ref
    
    # extract some basic features from the F-d response data
    sim_response = add_basic_features(sim_response)
    
    # get the stiffness-hardening features
    sim_features = get_SH_features(sim_response, K_lim=0.1)
    
    # calculate the force-based calibration error
    eps_CAL_df.loc[sim_id,'eps_CAL_F'] = calibration_force(ref_response, sim_response)
    
    # calculate the stiffness-hardening based calibration error
    eps_CAL_df.loc[sim_id,'eps_CAL_SH'] = calibration_stiffness_hardening(ref_features, sim_features)

print()
# review the calibration error dataframe
show_df(eps_CAL_df.describe())

1 2 3 4 5 6 7 8 9 


,eps_CAL_F,eps_CAL_SH
count,9.000000,9.000000
mean,0.285962,1.640207
std,0.158054,1.132297
min,0.121145,0.086925
25%,0.188285,0.196343
50%,0.210821,2.342655
75%,0.482478,2.410979
max,0.501994,2.487299


### Error in drifts in dynamic tests

In [29]:
# coming soon...